In [1]:
import struct

In [2]:
filename = "data/twitter_index"

In [48]:
file = open(filename, "r")

In [5]:
temp = struct.iter_unpack("I", file)

TypeError: a bytes-like object is required, not '_io.TextIOWrapper'

In [6]:
import numpy as np

In [49]:
numUsers = np.fromfile(file, dtype=np.uint32, count=1)[0]

In [39]:
file.close()

In [29]:
twitterID = np.dtype([("id", np.uint32), ("followers", np.uint32), ("followersPosition", np.uint64), ("friends", np.uint32), ("friendsPosition", np.uint64)])

In [50]:
users = np.fromfile(file, dtype=twitterID, count=-1)

In [51]:
file.close()

In [47]:
type([4,2]) == list

True

In [1]:
import twitterGraph as tg

In [2]:
graph = tg.twitterGraph()

In [3]:
graph.numUsers

41652229

In [5]:
len(graph.userData)

41652229

In [3]:
import random as rand

In [6]:
temp = graph.userData[rand.randint(0, graph.numUsers-1)]
graph.getFriends(temp)


AssertionError: 